In [1]:
import numpy as np


from astropy.coordinates import Angle
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy import wcs
from pyvo.dal import sia

In [2]:
DEF_ACCESS_URL = "https://datalab.noirlab.edu/sia/des_dr2" # DES SIA service URL
svc = sia.SIAService(DEF_ACCESS_URL)


ra, dec = 73.4708333, -50.3763611


# 4amin
imgTable = svc.search((ra,dec), (Angle(4 * u.arcmin), Angle(4 * u.arcmin)), intersect='CENTER', verbosity=0).to_table()
onlyImage = imgTable[(imgTable['prodtype']=='image') & \
                     ((imgTable['obs_bandpass'] == 'g DECam SDSS c0001 4720.0 1520.0'))].to_pandas()
print(onlyImage['access_url'][1])
url_1 = onlyImage['access_url'][1]


ra, dec = 11.6420833, -1.5823889


# 4amin
imgTable = svc.search((ra,dec), (Angle(4 * u.arcmin), Angle(4 * u.arcmin)), intersect='CENTER', verbosity=0).to_table()
onlyImage = imgTable[(imgTable['prodtype']=='image') & \
                     ((imgTable['obs_bandpass'] == 'g DECam SDSS c0001 4720.0 1520.0'))].to_pandas()
print(onlyImage['access_url'][1])
url_2 = onlyImage['access_url'][1]

https://datalab.noirlab.edu/svc/cutout?col=des_dr2&siaRef=DES0455-5040_r4939p01_g_nobkg.fits.fz&extn=1&POS=73.4708333,-50.3763611&SIZE=0.06666666666666667,0.06666666666666667
https://datalab.noirlab.edu/svc/cutout?col=des_dr2&siaRef=DES0045-0124_r4907p01_g_nobkg.fits.fz&extn=1&POS=11.6420833,-1.5823889&SIZE=0.06666666666666667,0.06666666666666667


In [3]:
from urllib.request import urlretrieve

urlretrieve(url_1, "cutout_1.fits")
urlretrieve(url_2, "cutout_2.fits")

('cutout_2.fits', <http.client.HTTPMessage at 0x7ffaec783190>)

In [4]:
def image_limits(hdr):
    """
    Function to get the image limites in wcs
    Input
     - hdr pyfits_hdr : image hdr
    Output
     -
    #FIXME - finish documentation
    """
    im_wcs = wcs.WCS(hdr, naxis=2)

    pixcrd = np.array([[0,0]], np.float_)

    coor1 = im_wcs.wcs_pix2world( [[0,0]], 1)
    coor2 = im_wcs.wcs_pix2world( [[hdr["NAXIS1"], hdr["NAXIS2"]]], 1)

    coor3 = im_wcs.wcs_pix2world( [[0, hdr["NAXIS2"]]], 1)
    coor4 = im_wcs.wcs_pix2world( [[hdr["NAXIS1"], 0]], 1)

    c0min = min([coor1[0][0], coor2[0][0], coor3[0][0], coor4[0][0]])
    c0max = max([coor1[0][0], coor2[0][0], coor3[0][0], coor4[0][0]])

    c1min = min([coor1[0][1], coor2[0][1], coor3[0][1], coor4[0][1]])
    c1max = max([coor1[0][1], coor2[0][1], coor3[0][1], coor4[0][1]])

    return [c0min, c0max], [c1min, c1max]

In [5]:
def get_cutout_distances(file):

    hdr = fits.open(file)[0].header

    ras, decs = image_limits(hdr)
    ra_min, ra_max = ras
    dec_min, dec_max = decs

    c1_dec = SkyCoord(ra_min*u.deg, dec_min*u.deg, frame='icrs')
    c2_dec = SkyCoord(ra_min*u.deg, dec_max*u.deg, frame='icrs')

    c1_rad = SkyCoord(ra_min*u.deg, dec_min*u.deg, frame='icrs')
    c2_rad = SkyCoord(ra_max*u.deg, dec_min*u.deg, frame='icrs')

    dec_distance = c1_dec.separation(c2_dec).arcmin
    rad_distance = c1_rad.separation(c2_rad).arcmin
    
    return [file, dec_distance, rad_distance]

In [6]:
get_cutout_distances('cutout_1.fits')

['cutout_1.fits', 2.5497095555969205, 2.5486326653867692]

In [7]:
get_cutout_distances('cutout_2.fits')

['cutout_2.fits', 4.002370649145821, 4.002240081910709]